In [2]:
!pip install onnxruntime


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install keras-image-helper


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:
import os

import numpy as np
import onnxruntime as ort
from keras_image_helper import create_preprocessor

In [11]:
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx

--2025-12-08 08:34:45--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/398ded4a-c41c-4e5a-9672-acb7e441de54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-08T04%3A19%3A53Z&rscd=attachment%3B+filename%3Dhair_classifier_v1.onnx.data&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-08T03%3A19%3A21Z&ske=2025-12-08T04%3A19%3A53Z&sks=b&skv=2018-11-09&sig=sothPY8Dseev7WhhFBP8LuXd5q%2FJ6k2CApEB6Kp9g1A%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTE2NjY4MywibmJmIjoxNzY1MTY0ODgzLCJwY

## Question 1

To be able to use this model, we need to know the name of the input and output nodes.

What's the name of the output:

- **output**
- sigmoid
- softmax
- prediction

In [13]:
model_name = 'hair_classifier_v1.onnx'

In [14]:
session = ort.InferenceSession(
    model_name, providers=["CPUExecutionProvider"]
)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

In [15]:
output_name

'output'

## Preparing the image

In [16]:
!pip install pillow


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


You'll need some code for downloading and resizing images. You can use this code:

In [17]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

## Question 2: Target size

Let's download and resize this image:

https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

Based on the previous homework, what should be the target size for the image?

- 64x64
- 128x128
- **200x200**
- 256x256

In [53]:
url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'

img = download_image(url)
img = prepare_image(
    img, 
    target_size=(200, 200)
)

## Question 3

Now we need to turn the image into numpy array and pre-process it.

Tip: Check the previous homework. What was the pre-processing we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

- -10.73
- **-1.073**
- 1.073
- 10.73

In [54]:
import torch
from torchvision import transforms
import numpy as np

preprocess = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

img_t = preprocess(img)
first_pixel_R_channel = img_t[0, 0, 0]
print(first_pixel_R_channel)

tensor(-1.0733)


## Question 4

Now let's apply this model to this image. What's the output of the model?

- **0.09**
- 0.49
- 0.69
- 0.89

In [55]:
def preprocess_pytorch_style(X):
    # X: shape (1, 299, 299, 3), dtype=float32, values in [0, 255]
    X = X / 255.0

    mean = np.array([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1)
    std = np.array([0.229, 0.224, 0.225]).reshape(1, 3, 1, 1)

    # Convert NHWC → NCHW
    # from (batch, height, width, channels) → (batch, channels, height, width)
    X = X.transpose(0, 3, 1, 2)  

    # Normalize
    X = (X - mean) / std

    return X.astype(np.float32)


preprocessor = create_preprocessor(
    preprocess_pytorch_style,
    target_size=(200, 200)
)

In [56]:
X = preprocessor.from_url(url)
result = session.run([output_name], {input_name: X})
result

[array([[0.09156627]], dtype=float32)]

## Prepare the lambda code
Now you need to copy all the code into a separate python file. You will need to use this file for the next two questions.

Tip: you can test this file locally with ipython or Jupyter Notebook by importing the file and invoking the function from this file.

## Docker
For the next two questions, we'll use a Docker image that we already prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.13

COPY hair_classifier_empty.onnx.data .
COPY hair_classifier_empty.onnx .
```

Note that it uses Python 3.13.

The docker image is published to agrigorev/model-2024-hairstyle:v3.

A few notes:

The image already contains a model and it's not the same model as the one we used for questions 1-4.

## Question 5
Download the base image agrigorev/model-2025-hairstyle:v1. You can do it with `docker pull`.

So what's the size of this base image?

- 88 Mb
- 208 Mb
- **608 Mb**
- 1208 Mb

You can get this information when running docker images - it'll be in the "SIZE" column.

In [57]:
!docker pull agrigorev/model-2025-hairstyle:v1

v1: Pulling from agrigorev/model-2025-hairstyle
Digest: sha256:9e43d5a5323f7f07688c0765d3c0137af66d0154af37833ed721d6b4de6df528
Status: Image is up to date for agrigorev/model-2025-hairstyle:v1
docker.io/agrigorev/model-2025-hairstyle:v1


In [58]:
!docker image ls | grep agrigorev/model-2025-hairstyle:v1

agrigorev/model-2025-hairstyle:v1    4528ad1525d5        608MB             0B        


## Question 6

Now let's extend this docker image, install all the required libraries and add the code for lambda.

You don't need to include the model in the image. It's already included. The name of the file with the model is `hair_classifier_empty.onnx` and it's in the current workdir in the image (see the Dockerfile above for the reference). The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

What's the output from the model?

- -1.0
- **-0.10**
- 0.10
- 1.0